In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import urllib3
import requests
import json
import urllib.request
import re
import time
import logging
import pandas
from collections import OrderedDict
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import sys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import StaleElementReferenceException
import math
import pandas as pd

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver', options=chrome_options)

In [ ]:
driver.get('https://www.wunderground.com/history/daily/us/ny/latham/KALB/date/2001-1-1')
time.sleep(3)
driver.find_element_by_id('wuSettings').click()
time.sleep(1)
driver.find_element_by_class_name('button-group').find_elements_by_class_name('button')[1].click()
time.sleep(3)

In [ ]:
zipcodes = {
    1201: 'https://www.wunderground.com/history/daily/us/ma/pittsfield/KPSF' #add only unique ones
}

In [ ]:
def get_hourly(link, year,mon,day):
    processed_data = []
    driver.get(link+'/date/'+str(year)+'-'+str(mon)+'-'+str(day))
    time.sleep(3)
    raw_data = driver.find_elements_by_class_name('mat-row.cdk-row.ng-star-inserted')
    summary = driver.find_element_by_tag_name('tbody').text.split('\n')
    
    day_entries.append({'day':str(year)+'-'+str(mon)+'-'+str(day),  
                        'entries':len(raw_data),
                        'historic_average_high': summary[0].split()[3],
                        'historic_average_low': summary[1].split()[3],
                        'record_high': summary[0].split()[4],
                        'record_low': summary[1].split()[4]})
    print(str(year)+'-'+str(mon)+'-'+str(day), len(raw_data))
    for i in raw_data:
        x = {}
        s = i.text.split()
        x['Zipcode'] = str(zc) if zc>10000 else '0'+str(zc)
        x['Date']=str(year)+'-'+str(mon)+'-'+str(day)
        x['Weather_Condition'] = ' '.join(s[17:])
        x['Time'] = s[0]+' '+s[1]
        x['Temperature'] = s[2]
        x['Dew Point'] = s[4]
        x['Humidity'] = s[6]
        x['Wind'] = s[8]
        x['Wind Speed'] = s[9]
        x['Wind Gust'] = s[11]
        x['Pressure'] = s[13]
        x['PRECIPITATION'] = s[15]
        processed_data.append(x)
    return processed_data

def get_zipcode_data(link):
    for i in range(2001,2007):
        for j in range(1,13):
            if j in [1,3,5,7,8,10,12]:
                for k in range(1,32):
                     for d in get_hourly(link, i,j,k):
                            hourly_data.append(d)
            if j in [4,6,9,11]:
                for k in range(1,31):
                    for d in get_hourly(link, i,j,k):
                            hourly_data.append(d)
            if j in [2]:
                if(i==2004):
                    for k in range(1,30):
                        for d in get_hourly(link, i,j,k):
                            hourly_data.append(d)
                else:
                    for k in range(1,29):
                        for d in get_hourly(link, i,j,k):
                            hourly_data.append(d)
        df = pd.DataFrame(hourly_data, columns = hourly_data[0].keys())
        print('writing data for '+str(zc)+' ' + str(i))
        df.to_excel("/content/drive/MyDrive/weather/"+str(zc) if zc>10000 else '0'+str(zc)+"_Raw_Hourly.xlsx", index=None)
        df = pd.DataFrame(day_entries, columns = day_entries[0].keys())
        print(str(zc)+' ' + str(i)+' complete. now writing file for entries per date')
        df.to_excel("/content/drive/MyDrive/weather/"+str(zc) if zc>10000 else '0'+str(zc)+"_Entries_Hourly.xlsx", index=None)

In [ ]:
for zc in zipcodes.keys():
    hourly_data = []
    day_entries = []
    get_zipcode_data(zipcodes[zc])